In [ ]:
import pandas as pd
import numpy as np
import psycopg2
from sqlalchemy import MetaData, create_engine, Table, Column, ForeignKey
from sqlalchemy import Integer, String, Date, Float, Time
from sqlalchemy import exc

In [ ]:
#Constantes
DB = "dbtest"
HOST = "localhost"
PORT = "5432"
PASS = "password"
USER = "postgres"

# Diagrama de DW  - Inmobiliaria - Esquema estrella

![Diagrama](img/Inmobiliaria.png "Diagrama")

In [ ]:
url_connect = USER + ':' + PASS + '@' + HOST+ ':' + PORT

try:
    engine = create_engine('postgresql://' + url_connect)
    conn = engine.connect()
    conn.execute("commit")
    conn.execute("create database " + DB)
    conn.close()
except Exception:
    print("La Base de Datos ya existe")

In [ ]:
engine = create_engine('postgresql://' + url_connect + "/" + DB)

In [ ]:
metadata = MetaData()


estate = Table('estate', metadata,
              Column('id', Integer, primary_key=True),
              Column('category', String),
              Column('area', Float, nullable=False),
              Column('city', String, nullable=False),
              Column('province', String, nullable=False),
              Column('rooms', Integer, nullable=False),
              Column('bedrooms', Integer, nullable=False),
              Column('garage', Integer),
              Column('meters', Float, nullable=False))

customer = Table('customer', metadata,
                Column('id', Integer, primary_key=True),
                Column('name', String, nullable=False),
                Column('surname', String, nullable=False),
                Column('budget', Float),
                Column('address', String),
                Column('city', String),
                Column('phone', String))

visit = Table('visit', metadata, 
             Column('id', Integer, primary_key=True),
             Column('duration', Integer, nullable=False))

sale = Table('sale', metadata, 
            Column('id', Integer, primary_key=True),
            Column('agreed_price', Float, nullable=False),
            Column('status', String, nullable=False))

fact = Table('fact', metadata,
            Column('id', Integer, primary_key=True),
            Column('id_estate', Integer, ForeignKey('estate.id')),
            Column('id_cust', Integer, ForeignKey('customer.id')),
            Column('date', Date, nullable=False),
            Column('id_visit', Integer, ForeignKey('visit.id'), nullable=True),
            Column('id_sale', Integer, ForeignKey('sale.id'), nullable=True))


In [ ]:
metadata.create_all(engine)

Carga de Datos
--------------

In [ ]:
try:
    df_estate = pd.read_csv('dataTp3/Inmobilaria/Estate.csv')
    df_estate.to_sql('estate', engine, index=None, if_exists="append")
    
except exc.IntegrityError:
    print("Los datos de Estate ya han sido cargados en la BD")

In [ ]:
try:
    df_customer = pd.read_csv('dataTp3/Inmobilaria/Customer.csv')
    df_customer.to_sql('customer', engine, index=None, if_exists="append")
except exc.IntegrityError:
    print("Los datos de Customer ya han sido cargados en la BD")  

In [ ]:
try:
    df_visit = pd.read_csv('dataTp3/Inmobilaria/Visit.csv')
    df_visit.to_sql('visit', engine, index=None, if_exists="append")
except exc.IntegrityError:
    print("Los datos de Visit ya han sido cargados en la BD")

In [ ]:
try:
    df_sale = pd.read_csv('dataTp3/Inmobilaria/Sale.csv')
    df_sale.to_sql('sale', engine, index=None, if_exists="append")
except exc.IntegrityError:
    print("Los datos de Sale ya han sido cargados en la BD")

In [ ]:
try:
    df_fact = pd.read_csv('dataTp3/Inmobilaria/Fact.csv')
    df_fact.to_sql('fact', engine, index=None, if_exists="append")
except exc.IntegrityError:
    print("Los datos de Fact ya han sido cargados en la BD")

# Consultas

## Consulta 1:

¿Qué tipo de propiedad se vendió por el precio más alto con respecto a cada
ciudad y meses?


In [ ]:
string_sql = """select city, date_part('month',"date") as mes, category, MAX(agreed_price) as Price_Max from fact
                inner join estate on (id_estate = estate.id)
                inner join sale on (id_sale = sale.id)
                where id_sale is not NULL
                group by city, category, mes
                order by city ASC; """

df_propiedades = pd.read_sql_query(string_sql,engine.connect()) 
df_propiedades

# Consulta 2:
¿Quén ha comprado un piso con el precio más alto con respecto a cada mes?

In [ ]:
string_sql = """select mes, name, surname, agreed_price from (
                    select mes, name, surname, agreed_price, rank() over particion_por_mes
                        from ( select date_part('month',"date") as mes, "name", surname, category, agreed_price
                            from fact inner join estate on (id_estate = estate.id)
                            inner join sale on (id_sale = sale.id) 
                            inner join customer on (id_cust = customer.id)
                        where id_sale is not NULL and category = 'piso'
                        order by agreed_price DESC) as pepe
                        WINDOW particion_por_mes AS (partition by mes order by agreed_price DESC)
                ) as sub_query
                WHERE rank = 1; """

df_clientes_pisos = pd.read_sql_query(string_sql,engine.connect()) 
df_clientes_pisos


## Consulta 3:
¿Cuál es la duración media de visitas en las propiedades de cada categoría?

In [ ]:
string_sql = """ select distinct category, tiempo_promedio
                 from (
                        select category, avg(duration)
                        over particion_por_categoria as tiempo_promedio
                        from (
                                fact inner join estate on (id_estate = estate.id) 
                                inner join visit on (id_visit = visit.id) )
                        WINDOW particion_por_categoria AS (partition by category)
                 ) as promedio; """
df_mayor_duracion = pd.read_sql_query(string_sql,engine.connect()) 
df_mayor_duracion

##Tiempo promedio medido en horas